In [1]:
import numpy as np
import random

In [2]:
data = open(r'C:\Users\harla\Desktop\Jupyter Notebooks\RNN basic\dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
print(sorted(chars))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
There are 19909 total characters and 27 unique characters in your data.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
#ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
ix_to_char = {}
for i,ch in enumerate(sorted(chars)):
    ix_to_char[(i)] = ch
print(ix_to_char)
print("Total charachters are :", len(ix_to_char))
#print(type(char_to_ix))

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
Total charachters are : 27


In [4]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [5]:
def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

In [6]:
def initialize_parameters(n_a, n_x, n_y):
    
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

In [7]:
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

In [8]:
def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # probabilities for next chars 
    
    return a_next, p_t

In [9]:
def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    loss = 0
    
    for t in range(len(X)):
        
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache


In [10]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

In [11]:
def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a


In [12]:
def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

In [13]:
def clip(gradients, maxValue):
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -1 * maxValue, maxValue, out = gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [14]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    gradients = clip(gradients, maxValue = 5)
    
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    
    return loss, gradients, a[len(X)-1]

In [15]:
def sample(parameters, char_to_ix):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    x = np.zeros(shape = (vocab_size,1))
    a_prev = np.zeros(shape = (n_a,1))
    
    indices = []
    
    idx = -1 
    
   
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        a = np.tanh(np.dot(Wax,x) + np.dot(Waa,a_prev) + b)
        z = np.dot(Wya,a) + by
        y = softmax(z)
        
        
        idx = np.random.choice(list(range(vocab_size)),  p = y.ravel())

        indices.append(idx)
        
        x = np.zeros(shape = (vocab_size,1))
        x[idx] = 1
        
        a_prev = a
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [16]:
def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

In [17]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    
    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open(r'C:\Users\harla\Desktop\Jupyter Notebooks\RNN basic\dinos.txt') as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    for j in range(num_iterations):
        
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        #print(examples[index])
        #print(X)
        Y = X[1:] + [char_to_ix["\n"]]
        
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        
        loss = smooth(loss, curr_loss)

        if j %  500 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            for name in range(dino_names):
                
                sampled_indices = sample(parameters, char_to_ix)
                print_sample(sampled_indices, ix_to_char)
      
            print('\n')
        
    return parameters

In [18]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.084039


Duswkmyhfavhehbomhmsshibhsiygn
Udakpikuktonvjrrqjhywfoheazlooihhrmvrzaahztzbqd
Crcrsuljvhwxyjepygveviyrjssuwnphopffntkuxesbiwdrdyrgh
Fxocwzceh
Yzgqluorzhronaagevoxvnut
Zezpzsuiwgbmyifveoohkti


Iteration: 500, Loss: 28.062455

Hlorurusaxrurus



Ror
Xhbocnupaoreaaratagruras
Zmo


Iteration: 1000, Loss: 28.907741

Zzmohgsaurus
Es
Us
Thosapillnjxurusairteprasaerur
Lhonicaaonil
Paurosasaurus
Miptvuaurus


Iteration: 1500, Loss: 28.684192


Ratrus
A
Blareoporodanhirus
Aus
Ycyus
Irumohus


Iteration: 2000, Loss: 27.842364

Dotosaurus
Annorourus
Sauritosaurus
Ryalthennbresansaurus
Rusnprasaurus
Tamipsaurus
A


Iteration: 2500, Loss: 27.324245

Ocerbelon
Aleromosaurus
Iadtodicochucoga
Tonourus
Yrueragonhaugusterpa
Tonbocrohotoschhabamosaurus
Autosaurus


Iteration: 3000, Loss: 26.794676

Chutolitganytanchus
Engataulinongakmodlunophisaurus
Rdenmasaurus
Antorasaurus
Anta
Ciceng
Andorattansaurus


Iteration: 3500, Loss: 26.187059

Aadosaurpancoryteroreansaurus
Hos

Iteration: 31000, Loss: 22.462700

Belaranitan
Verangasaurus
Uarmia
Diteqatops
Ricelalosaurus
Tescosaurus
Abrosaurus


Iteration: 31500, Loss: 22.392385

Weciaeavosaurus
Abpalong
Iumosaurus
Adiliasaurus
Hitioradosaurus
Cancrtha
Criidon


Iteration: 32000, Loss: 22.521584

Phynosaurus
Apathustanewhansaurus
Opelta
Seirasaurus
Machilosaurosaurus
Rastegonsaurus
Paodon


Iteration: 32500, Loss: 22.403131

Celopirogsaurus
Renochisaurus
Gaeiconyxeosaurus
Ylnykosaurus
Deriiops
Kelomagotor
Saus


Iteration: 33000, Loss: 22.404179

Igaporosaurus
Anousaurus
Amsoxlesaurus
Ropinnosaurus
Coplkbisymimpicerovennasaurushyphyposaurus
Burtiseus
Canpodon


Iteration: 33500, Loss: 22.530037

Entanoceranosaurus
Wenydratisaurus
Iclopudameodiptor
Rgpanosaurus
Ryinatesaurus
Tanchinospurosaurus
Anati


Iteration: 34000, Loss: 22.423785

Hulonatops
Ertellegnogalips
Ikathostacalagosaurus
Toranodudopaceratoratous
Saloromalliteomi
Liangactaronliroma
Kicelatops


Iteration: 34500, Loss: 22.392548

Membosaurus
Ggytho